# Introduction to Natural Language Processing with spaCy


## What is NLP?

Natural language processing is a way for computers to analyze, understand, and derive meaning from human language. With appropriate use and organization, NLP can be used to help developers perform a variety of tasks, including summarization, translation, named entity recognition, relationship extraction, sentiment analysis, speech recognition, and topic segmentation for a given dataset or group of texts.

## What can we use it for?

Things like chat bots, voice-to-text software, and customer service sentiment analysis are examples of applications of NLP. The use of (and development of tools for) NLP has experienced rapid growth over the last decade and is currently being integrated into a variety of fields. 

For example, a retailer on Amazon may run a sentiment analysis on all comments for a certain product. This could reveal general attitudes towards both the company and the item in question, ultimately leading to improvements and adjustments. On another hand, Siri, Apple's personal voice assistant, is the almagamation of years of work in NLP - Siri can recognize, conceptualize, and respond to a wide array of questions and comments. This not only improves the iPhone user experience in general but also increases the accessibility of the product.

## Basic Terminology

__Corpus__ (plural: corpora) is defined as a large collection of liguistic data. In other words, corpora serve as our datasets, or our informaiton to process and train models on.

__Tokenization__ is the process of segmenting text into words, punctuation marks, etc. This is one of the first steps in processing the text into workable components.

__Part-of-speech (POS) tagging__ involves assigning word types (parts of speech) to tokens, like _verb_, _noun_, _preposition_, etc. 

__Dependency Parsing__ is the process of assigning syntactic dependency labels that describe the relations between individual tokens. For example, in the sentence _The dog ran through the park_, dependency parsing would recognize that _dog_ is the subject of the sentance, _ran_ is the main verb, and so on and so forth.

__Lemmatization__ is defined as assigning the base form of words. For example, the lemma of “was” is “be”, and the lemma of “rats” is rat.

__Sentence Boundary Detection (SBD)__ is responsible for finding and segmenting individual sentences within a text. 

__Named Entity Recognition (NER)__ involves labelling named “real-world” objects, like persons, companies, or locations. For example, in some instances, we want "Amazon" to be recognized as an electronic company as opposed to a forest.

__Similarity__ is the process of comparing words, text spans, and documents to see how similar they are to each other. This is helpful when trying to determine a common theme, or, on the other hand, divides between content.

__Text Classification__ assigns categories or labels to a whole document or parts of a document. 


## What is spaCy?

+ Free, open-source software
+ Designed for advanced, industrial strength natural language processing in python

In [1]:
import spacy
from spacy import displacy

We begin by importing the necessary packages, `spaCy` being the most notable. We then load the English stasticial model (which is set up as its own python package) - this is the main program we will be working with. Using a model for a specific language enables spaCy to predict linguistic annotations – for example, whether a word is a verb or a noun. Though some of spaCy's features are available without a language model, most of its functions require one. 

In [2]:
nlp = spacy.load('en')

First, we'll walk through some of the basic functions of spaCy, based on the definitions above. We begin by reading in a few simple sentences, just to get a feel for how the package works.

In [3]:
doc = nlp(u'I\'m having such a wonderful day! It is sunny out and there are flowers. Do you want to get some ice cream?')

## Tokenizing

When we read something in with `nlp`, spaCy automatically tokenizes it - for example, it breaks _I'm_ into _I_ and _'m_, and each word is its own element. The object `doc` can be indexed to access individual tokens. We can also view all the individual sentences from the paragraph by iterating through the `sents` property.

In [4]:
print(doc[0])
print(doc[1])
print(doc[5])
print(doc[7])

for sent in doc.sents:
    print(sent)

I
'm
wonderful
!
I'm having such a wonderful day!
It is sunny out and there are flowers.
Do you want to get some ice cream?


## Visualization

SpaCy has a built-in visualization package called `displaCy` that plots sentence dependencies, entity recognition, and more.

### Dependency

Dependency parsing is the process of analyzing a sentence and assigning a syntactic structure to it: labeling the subject, the verb, etc., and how the different elements of the sentence depend on one another.

In [5]:
doc = nlp('The quick brown fox jumped over the lazy dog.')
options={'distance':100}
displacy.render(doc, style='dep', jupyter=True, options=options)

### Entity Recognition

In [6]:
doc2 = nlp("TD Ameritrade, ProQuest, Google, Domino's and the University of Michigan are companies that hire data scientists in Ann Arbor")
colors = {'GPE': 'linear-gradient(0deg, #deebf7, #3182bd)',
         'ORG': 'linear-gradient(90deg, #fee6ce, #e6550d)'}
options = {'colors': colors}
displacy.render(doc2, style='ent', jupyter=True, options=options)

**Tip**: For more visualizer options, see https://spacy.io/api/top-level#displacy_options

### Manual Entity Recognition

spaCy also offers the option of defining your own entity recognition:

In [7]:
ex = [{'text': 'But Google is starting from behind.',
       'ents': [{'start': 4, 'end': 10, 'label': 'ORG'}],
       'title': 'This is my title'}]
displacy.render(ex, style='ent', jupyter=True, manual=True)

### spaCy Explain Method

Since spaCy is filled with plenty of useful tools, it's easy to lose track of what all the different abbreviations stand for. The spacy.explain() method can be used to get the description for the string representation of an entity label. For example, spacy.explain("LANGUAGE") will return "any named language".

In [8]:
part_of_speech = ['DET','VERB','NOUN']
for pos in part_of_speech:
    print(pos, spacy.explain(pos))

DET determiner
VERB verb
NOUN noun


### Word Count

In [9]:
from collections import Counter
w = (token.text for token in doc if token.is_stop == False and token.is_punct == False and token.text not in ['\n'] and token.prefix_ != '$')
freq = Counter(w)
freq.most_common(20)

[('The', 1),
 ('lazy', 1),
 ('quick', 1),
 ('brown', 1),
 ('dog', 1),
 ('jumped', 1),
 ('fox', 1)]

# Topic Modeling

One branch of NLP is topic modeling - a topic model is a kind of statistical model that is used to uncover the abstract topics and concepts that occur in a collection of documents. Topic modeling is frequently used to discover semantic structures in a text body, and as a data-mining tool to better understand large collections of data.

Now that we've seen how spaCy processes short text segments, let's explore what happens (and what we can work with) when we examine a much larger document. To work with topic modeling, we'll begin by using spaCy to tokenize a text file containing US news articles.

In [10]:
import os
wdir = r'C:\Users\epmarie\Downloads'

In [11]:
%%time
with open(os.path.join(wdir,'en_US.news.txt'),'r', encoding='utf-8') as fin:
    data = fin.read()
data = data[:900000]

Wall time: 1.15 s


In [12]:
%%time
news = nlp(data)

Wall time: 1min 24s


As with any large text, we can't start analysis until we clean the data. This invovles removing unimportant words and punctuation, begin by removing __stopwords__, which are commonly used words that have little value in determining sentiment or analyzing a document. Filtering out words like ‘the’, ‘is’, and ‘are’ helps speed up processes and helps keep the data clean while allowing us to focus on more significant/rarer terms. Luckily, each token has the built-in property `is_stop` to indicate whether or not it is considered a stopword.

Sometimes, in addition to removing stopwords, we'll want to remove punctuation from a piece. We may also choose to convert all words to lowercase or standardize dates and times. Note that the cleaning process will not be the same for every text or even for every analysis of the same text - sometimes we care about capitalization and punctuation as part of the sentiment and topic modeling analysis. For now, though, we'll remove the punctuation. We'll also remove any newline and possessive characters.

In [13]:
text=[]
for sentence in news.sents:
    text.append([token.text for token in sentence 
                 if token.is_punct == False and token.is_stop == False and token.text not in ["\n","'s"]])

## Gensim

Gensim is a powerful vector space modeling and topic modeling toolkit, commonly used for a variety of NLP tasks. Now that we've prepared our information for analysis, we'll use Gensim to perform topic modeling. 

In [14]:
from gensim import corpora, models

C:\Users\Public\Desktop\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


We begin by creating dictionary containing (key,value) pairs where `key` represents `word` and `value` represents `integer id`.

In [15]:
dictionary = corpora.Dictionary(text)
dictionary.token2id

{'Jug': 11116,
 'bearing': 8630,
 'Do': 3628,
 'appetizer': 21390,
 'mumbling': 17544,
 'Passion': 13605,
 'flyover': 13242,
 'Sponsored': 17751,
 '38-Davis': 16400,
 'determined': 4668,
 'divorced': 2141,
 'Jahmell': 12909,
 'Drury': 3286,
 'rental': 18621,
 "O'Loughlin": 6766,
 'liqueur': 21683,
 'conservative': 139,
 'minimum': 17102,
 'Malcolm': 10006,
 'beloved': 5658,
 'assumption': 17011,
 'guides': 19725,
 'Morris': 7398,
 'sights': 2755,
 'brisk': 5821,
 'spiritual': 9916,
 'Arrieta': 822,
 'KMBC': 12892,
 'nalusup@gmail.com': 5984,
 'sophomore': 6534,
 'Tryon': 17753,
 'connect': 15970,
 'underpinning': 14451,
 'Shrewsbury': 11634,
 'piddling': 16968,
 'energized': 1993,
 'condemned': 11643,
 'coveted': 18141,
 'Mooney': 11963,
 'anytime': 9591,
 'defended': 5942,
 'bishop': 11395,
 'steppes': 11328,
 'racist': 3897,
 'Winnetka': 19716,
 'Bentiu': 19262,
 '52': 6803,
 'minds': 14221,
 'downloading': 4047,
 'Treasurer': 9857,
 'Dippin': 11805,
 'Abbey': 19025,
 'Doan': 1009,
 

The function doc2bow() simply counts the number of occurrences of each distinct word, converts the word to its integer word id and returns the result as a sparse vector.

In [16]:
corpus = [dictionary.doc2bow(txt) for txt in text]

Latent Dirichlet allocation (LDA) is a generative statistical model. It is a pattern recognition and machine learning technique that works to find a linear combination of features that characterizes two or more classes of objects. We use it here to try to identify topics in our corpus.

In [17]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=30)

Below, we call `show_topics` to view the top 10 most significant words for the top 10 most significant topics that the LDA model has discovered/created. The factors in front of the texts represents each word's impact on the topic it is associated with.

In [18]:
lda.show_topics()

[(14,
  '0.018*"A" + 0.016*"3" + 0.012*"1" + 0.012*"second" + 0.011*"said" + 0.008*"The" + 0.008*"areas" + 0.008*"0" + 0.007*"4" + 0.007*"Washington"'),
 (1,
  '0.017*"The" + 0.011*"American" + 0.011*"This" + 0.008*"left" + 0.008*"All" + 0.008*"People" + 0.008*"black" + 0.007*"clear" + 0.007*"Richard" + 0.006*"reform"'),
 (28,
  '0.016*"said" + 0.015*"I" + 0.014*"n\'t" + 0.012*"It" + 0.011*"person" + 0.009*"comes" + 0.008*"years" + 0.008*"time" + 0.008*"store" + 0.008*"college"'),
 (23,
  '0.012*"team" + 0.010*"said" + 0.009*"The" + 0.009*"But" + 0.008*"nation" + 0.008*"place" + 0.007*"Now" + 0.007*"taken" + 0.007*"like" + 0.006*"Steve"'),
 (4,
  '0.039*"I" + 0.032*"said" + 0.018*"We" + 0.016*"\'ve" + 0.012*"better" + 0.011*"got" + 0.011*"schools" + 0.009*"\'m" + 0.009*"school" + 0.009*"thought"'),
 (7,
  '0.017*"That" + 0.012*"said" + 0.010*"The" + 0.009*"use" + 0.009*"short" + 0.009*"president" + 0.008*"percent" + 0.007*"Obama" + 0.007*"days" + 0.006*"trip"'),
 (24,
  '0.010*"p.m." +

From these buzzwords, we can infer what general concept each "topic" is talking about. Take a minute to write down that you think each collection of keywords represents.

## Visualizations for Topic Modeling

Now that we have our topics and our keyword collections, we can present them in a visualization to get a different view on how important each topic is to the overall document, and how closely these topics are related.

In [19]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

pyLDAvis is compatible with gensim, scikit-learn, and GraphLab Create. Here is how you would use it with gensim. We need a gensim LDA model, corpus and dictionary - we'll use the ones we just built.

In [20]:
viz = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
viz

C:\Users\Public\Desktop\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
11     4.506510        1       1 -0.204128 -0.074559
4      4.024763        1       2 -0.162094  0.086083
14     3.901883        1       3  0.070006  0.123810
25     3.806003        1       4 -0.138035 -0.015080
17     3.558685        1       5  0.037054  0.120871
28     3.471254        1       6 -0.122522 -0.004628
19     3.465431        1       7 -0.031683 -0.030713
3      3.455828        1       8  0.100799  0.023682
12     3.374525        1       9 -0.036427  0.015554
7      3.374189        1      10 -0.024200 -0.018324
8      3.369915        1      11  0.077861 -0.090514
29     3.320177        1      12 -0.012358  0.031566
13     3.308756        1      13  0.068967 -0.058019
27     3.272860        1      14  0.103271 -0.088260
6      3.231877        1      15 -0.008254 -0.063755
9      3.230861        1      16 -0.010876  0.052428
2      3.225497        1      17 -0.025933 -0.003039
26     3.198782        1      18 -0.020691 -0.093652
5      3.173036        1      19  0.002329  0.058470
23     3.168443        1      20 -0.029173 -0.007547
0      3.163734        1      21  0.071448 -0.032284
15     3.148949        1      22  0.068976  0.005957
22     3.141309        1      23  0.001719  0.082049
21     3.137541        1      24  0.040507  0.029872
18     3.070582        1      25  0.009853 -0.007449
20     3.066095        1      26  0.045585  0.086078
16     3.028462        1      27  0.060087  0.004993
24     3.016915        1      28 -0.007085  0.026768
10     2.929758        1      29  0.041565 -0.015742
1      2.857381        1      30  0.033432 -0.144616, topic_info=      Category        Freq         Term       Total  loglift  logprob
term                                                                 
5458   Default  747.000000            I  747.000000  30.0000  30.0000
4953   Default  352.000000            $  352.000000  29.0000  29.0000
10072  Default  370.000000          n't  370.000000  28.0000  28.0000
2027   Default  117.000000      million  117.000000  27.0000  27.0000
12806  Default  202.000000           We  202.000000  26.0000  26.0000
2131   Default  895.000000         said  895.000000  25.0000  25.0000
11264  Default  107.000000          old  107.000000  24.0000  24.0000
2079   Default  126.000000      percent  126.000000  23.0000  23.0000
13854  Default   88.000000         know   88.000000  22.0000  22.0000
8239   Default   90.000000        think   90.000000  21.0000  21.0000
14690  Default   83.000000          've   83.000000  20.0000  20.0000
21934  Default  128.000000       season  128.000000  19.0000  19.0000
19610  Default  153.000000            A  153.000000  18.0000  18.0000
21495  Default  845.000000          The  845.000000  17.0000  17.0000
13331  Default  258.000000           He  258.000000  16.0000  16.0000
18270  Default   79.000000           'm   79.000000  15.0000  15.0000
12945  Default  107.000000         That  107.000000  14.0000  14.0000
4476   Default   71.000000            3   71.000000  13.0000  13.0000
5236   Default   65.000000          n’t   65.000000  12.0000  12.0000
17596  Default   53.000000           So   53.000000  11.0000  11.0000
17639  Default  178.000000         like  178.000000  10.0000  10.0000
15519  Default   74.000000          St.   74.000000   9.0000   9.0000
3029   Default   81.000000          New   81.000000   8.0000   8.0000
8851   Default   86.000000        There   86.000000   7.0000   7.0000
19326  Default   52.000000       School   52.000000   6.0000   6.0000
6949   Default  135.000000        going  135.000000   5.0000   5.0000
5470   Default   94.000000       school   94.000000   4.0000   4.0000
21252  Default  182.000000        years  182.000000   3.0000   3.0000
622    Default  149.000000            1  149.000000   2.0000   2.0000
15506  Default   38.000000          boy   38.000000   1.0000   1.0000
...        ...       

Each bubble on the large plot represents a topic. The larger the bubble, the more frequently that topic is referenced. Ideally, we want large topic bubbles that are well separated and do not overlap with one another. A model with too many topics will have many overlaps and will be made up of many small sized bubbles clustered in one region of the chart. It appears that this particular dataset (with this analysis) doesn't have as clear separations as we'd like, and falls into the "too many topics" category.

The visualization is also interactive - if you hover over a bubble, it highlights the terms that the topic includes, and shows their frequency (both overall and within the selected topic).

Visit https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/ for additional resources and work with topic modelin in Gensim.

## Sentiment Analysis

TextBlob is another NLP library in Python. Unlike spaCy, however, it is built entirely in Python, which docks its performance speed and high-processing ability a bit. TextBlob is a bit of a "best of the best" toolkit - it pulls some of the most effective and useful methods from packages like NLTK and Pattern. Here, we'll use it to perform sentiment analysis.

In [21]:
from textblob import TextBlob

In [22]:
#variety = nlp(u'I have never been to Paris, but I would love to go! How often do you travel there?')
flowers = TextBlob('These flowers are beautiful, they brighten up the place so much! I really love them.')
angry = TextBlob('You make me so angry; I just can\'t stand it.')

The `sentiment` attribute of a TextBlob "returns a tuple of form (polarity, subjectivity ) where polarity is a float within the range [-1.0, 1.0] and subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective."

Let's break this into its two parts: polarity and subjectivity. Polarity analysis takes into account the amount of positive or negative terms that appear in a given sentence - words like "beautiful" and "brighten" likely contribute to a higher score, while words like "angry" and "can't" would shift the polarity in a negative direction. Subjectivity, on the other hand, is almost like an error bound on the sentiment analysis. The subjectivity of words and phrases may depend on their context and an objective document may contain subjective sentences. Words and phrases that are labeled as more subjective may shift their polarity depending on their context, making it more difficult to determine the true sentiment of the phrase.

In [23]:
print(flowers.sentiment)
print(angry.sentiment)

Sentiment(polarity=0.675, subjectivity=0.8)
Sentiment(polarity=-0.5, subjectivity=1.0)


To get even _more_ specific, we can use the `sentiment_assessments` feature to view which words contributed to the polarity and subjectivity scores.

In [24]:
print(flowers.sentiment_assessments)
print(angry.sentiment_assessments)

Sentiment(polarity=0.675, subjectivity=0.8, assessments=[(['beautiful'], 0.85, 1.0, None), (['much', '!', 'really', 'love'], 0.5, 0.6, None)])
Sentiment(polarity=-0.5, subjectivity=1.0, assessments=[(['angry'], -0.5, 1.0, None)])


## Exercise

Test out a couple of different sentences and examine their sentiment analysis output. Try to write:
* A sentence that scores a neutral polarity (aim for a range in [-0.2, 0.2])
* A sentence that scores as highly _objective_
* A sentence that scores as highly _subjective_

In [25]:
net = TextBlob('I mean, it\'s pretty normal; I guess it\'s okay enough.')
net.sentiment_assessments

Sentiment(polarity=0.11750000000000001, subjectivity=0.6675, assessments=[(['mean'], -0.3125, 0.6875, None), (['pretty'], 0.25, 1.0, None), (['normal'], 0.15, 0.6499999999999999, None), (['okay'], 0.5, 0.5, None), (['enough'], 0.0, 0.5, None)])

In [26]:
obj = TextBlob('I really don\'t know...')
obj.sentiment_assessments

Sentiment(polarity=0.2, subjectivity=0.2, assessments=[(['really'], 0.2, 0.2, None)])

In [27]:
sub = TextBlob('Things are getting really funny, kind of interesting...')
sub.sentiment_assessments

Sentiment(polarity=0.45, subjectivity=0.7999999999999999, assessments=[(['really', 'funny'], 0.25, 1.0, None), (['kind'], 0.6, 0.9, None), (['interesting'], 0.5, 0.5, None)])

### Sentiment Analysis for US News

Now that we've seen how TextBlob sentiment analysis works on a few lines of text, we can see what polarity and subjectivity scores it assigns to the news dataset that we've been working with.

In [28]:
news_txt = TextBlob(data)
news_txt.sentiment

Sentiment(polarity=0.10219093467161112, subjectivity=0.43409946705408897)